1. Happy Number. Given a number, take square of all digits, and add them. If sum is 1, HAPPY, if it cycles, not.
2. Construct the largest number from an array.
3. Fast power. Comute $x^n$, _quickly_.
4. Longest increasing subsequence.
5. Skyline Problem.
6. Jump Game III -- **Always consider BFS for a graph problem!**
7. Jump Game IV -- **Always consider BFS for a graph problem!**

---

# 1. Happy Number?

Can you determine, algorithmically, if it will be a happy number?

If the same pattern of digits repeats, discounting the number of zeros, then it will repeat endlessly ...

Can you prove that this will always terminate?
* For the largest 3 digit number `999`, the next number is `243`
* For the largest 4 digit number `9999`, the next number is `324`
* For the largest 10 digit number `9999....`, the next number if `810`

Eventually, the number will come down, regardless of where is starts, and it will either terminate, or cycle.

In [4]:
class HappyNumber:
    def solve(self, number):
        if number == 0:
            return False
        
        def sumsq(n):
            a = 0
            while n != 0:
                a += (n%10)**2
                n = n//10
            return a
        
        seen = set()
        def happy(n):
            x = sumsq(n)
            if x == 1:
                return True
            elif x in seen:
                return False
            else:
                seen.add(x)
                return happy(x)
            
        return happy(number)

In [8]:
o = HappyNumber()
o.solve(101)

False

# 2. Construct the largest number from an array.

Create a number by concatenating array elements, in any order.

One way is to consider all `n!` permutations.

But you know, a number starting from 9 is greater than a number starting from 1.

So, start by sorting numbers in reverse lex order -- everything beginning with `9*` is first, then `8*` and so on ...

In [30]:
class ConstructLargest:
    def solve(self, numbers):
        lex = list(sorted([str(n) for n in numbers], reverse=True))
        for i in range(len(lex)-1):
            first = lex[i]
            second = lex[i+1]
            if first + second < second + first:
                lex[i], lex[i+1] = second, first
                
        return int(''.join(lex))

In [31]:
o = ConstructLargest()
o.solve([9, 991, 8])

99918

In [20]:
'99' > '991'

False

In [21]:
'99991' > '99199'

True

In [19]:
'30' > '33'

False

# 3. Fast Power.

Compute $x^n$

In [32]:
class FastPower:
    def solve(self, x, n):
        assert n >= 1
        
        def power(e):
            if e == 1:
                return x
            else:
                half = power(e//2)
                if e%2 == 0:
                    return half * half
                else:
                    return half * half * x
                
        return power(n)

In [36]:
o = FastPower()
o.solve(2, 11)

2048

# 4. Longest Increasing Subsequence.

Given an array of randomly ordered number, find the length of the longest increasing subsequence.
```
[3, 4, 5, 1] => 3
[1, 6, 17, 2, 3, 15, 7] => 4
```

In [43]:
class LongestIncSub:
    def solve(self, nums):
        def sar(x, array):
            if len(array) == 1:
                array[0] = x
            else:
                start = 0
                end = len(array) - 1
                while start <= end:
                    mid = start + (end - start)//2
                    if array[mid] >= x:
                        end = mid - 1
                    else:
                        start = mid + 1
                        
                array[end+1] = x
        
        temp = []
        for n in nums:
            if temp:
                if n == temp[-1]:
                    pass
                elif n > temp[-1]:
                    temp.append(n)
                else:
                    # Search and replace.
                    sar(n, temp)
            else:
                temp.append(n)
                
        print(temp)
        return len(temp)

In [46]:
o = LongestIncSub()
print(o.solve([3, 4, 5, 1]))
print(o.solve([1, 6, 17, 2, 3, 15, 7]))

[1, 4, 5]
3
[1, 2, 3, 7]
4


# 5. Skyline Problem

Given building `(start, end, height)` triples, can you build a skyline? I.e. the (x, y) coordinate pairs where the height changes.

You are encountering buildings.

Every time you encounter a new building's start or end, you need to query the highest point at the moment.
And that will become the outline.

In [60]:
from collections import namedtuple

class Skyline:
    def solve(self, buildings):
        skyline = []
        tracker = []
        Building = namedtuple("building", ["start", "end", "height"])
        buildings = [Building(*b) for b in buildings]
        
        Point = namedtuple("point", ["position", "height", "building", "isStart"])
        data = sorted([Point(b.start, b.height, ix, True) for ix, b in enumerate(buildings)] +\
                      [Point(b.end, b.height, ix, False) for ix, b in enumerate(buildings)], 
                      key=lambda b:b.position)
        
        for p in data:
            if p.isStart:
                tracker.append(p)
                best = max([b.height for b in tracker])
            else:
                best = max([b.height for b in tracker if b.building != p.building] + [0])
                
            if skyline:
                if skyline[-1] != best:
                    skyline.append(best)
            else:
                skyline.append(best)
                
        return skyline

In [61]:
o = Skyline()
o.solve([(1, 3, 4), (2, 6, 8), (5, 10, 2), (12, 13, 1), (13, 14, 2)])

[4, 8, 4, 8]

In [63]:
1 + float('inf')

inf

6 qualifying courses + 1 elective

Any two SCS courses may count as qualifying.

1 course from each area.

1st sem:
* Advanced NLP (11-711): qualifying and Human Language area
* Intro to ML (10-701): qualifying and Machine Learning area
* Statistics (36-700): elective
* Elective done, 2/6 qualifying done, 1 SCS qualifying left

2nd Sem:
* PGM (10-708): qualifying, SCS
* Convex Optimization: extra
* Multimodal: qualifying
* 4/6 qualifying done, 0 SCS left, Applications area left

3rd Sem:
* Speech Processing 11-751: qualifying and Applications area
* Langauge And Statistics 11-611: qualifying

# 6. Jump Game III

https://leetcode.com/problems/jump-game-iii/

You are given an array of non-neg numbers and a starting location. From each position `i`, you can move `array[i]` steps to the left or to the right. Return true if you can eventually end in any position `j` where `array[j]` is 0.

In [71]:
from functools import lru_cache

class JumpGame3:
    def solve(self, arr, start):
        # sink = set([i for i, n in enumerate(arr) if n == 0])
        @lru_cache(maxsize=None)
        def search(i):
            if i >= len(arr) or i < 0:
                return False
            elif arr[i] == 0:
                return True
            else:
                val = arr[i]
                arr[i] = None
                for temp in [i+val, i-val]:
                    if temp >= 0 and temp < len(arr) and arr[temp] is not None and search(temp):
                        arr[i] = val
                        return True
                    
                arr[i] = val
                return False
            
        return search(start)

In [72]:
o = JumpGame3()
print(o.solve([4,2,3,0,3,1,2], 5))
print(o.solve([4,2,3,0,3,1,2], 0))

True
True


In [76]:
print(o.solve([3,0,2,1,2], 2))

False


In [83]:
from collections import deque

class JumpGame3BFS:
    def solve(self, arr, start):
        seen = set()
        q = deque([start])
        while q:
            i = q.popleft()
            if i >= 0 and i < len(arr) and i not in seen:
                if arr[i] == 0:
                    return True
                else:
                    seen.add(i)
                    q.append(i + arr[i])
                    q.append(i - arr[i])
                    
        return False

In [86]:
o = JumpGame3BFS()
print(o.solve([4,2,3,0,3,1,2], 5))
print(o.solve([4,2,3,0,3,1,2], 0))

True
True


In [85]:
print(o.solve([3,0,2,1,2], 2))

False


# 7. Jump Game IV

You can move 1 step right, left OR jump to another index which has the same value as the current one.

### **NEWSFLASH!!**
* You CANNOT apply DFS and Memo. The cache will store the first answer it finds, which might not be the best answer.
* Also, you cannot X-out nodes for skipping in the future -- the answer might start from the previous node that falls on the `i-1` (i.e. *left*) path.

BFS on the otherhand, gives you the same run-time, but is guaranteed to return the shortest path EVERY SINGLE TIME. Becuase that is the nature of the algorithm. 

### Another Newsflash
* The solution you have written gets a Memory Exceeded issue if/when there are a lot of consecutive numbers -- the graph and the queue size becomes huugeee -- you need to eliminate multiple vals: `[7, 7, 7, 7, 11] ==> [7, 7, 11]`
* https://leetcode.com/problems/jump-game-iv/discuss/1055395/Python-BFS-w-comment

In [90]:
from collections import deque

class JumpGame7:
    def solve(self, arr):
        jumps = {}
        for ix, v in enumerate(arr):
            if v in jumps:
                jumps[v].add(ix)
            else:
                jumps[v] = set([ix])
                
        graph = {}
        for ix, v in enumerate(arr):
            if ix not in graph:
                graph[ix] = set()
                
            if ix + 1 < len(arr):
                graph[ix].add(ix+1)
            if ix - 1 >= 0:
                graph[ix].add(ix-1)
            for j in jumps[v]:
                if ix != j:
                    graph[ix].add(j)
                    
        q = deque([(0, 0)])  # Ix, Step
        seen = set()
        while q:
            ix, steps = q.popleft()
            if ix == len(arr)-1:
                return steps
            else:
                if ix not in seen and ix in graph:
                    seen.add(ix)
                    nbrs = [n for n in graph[ix] if n not in seen]
                    del graph[ix]
                    for n in nbrs:
                        q.append((n, steps+1))
                        
        return None

In [91]:
o = JumpGame7()
print(o.solve([100,-23,-23,404,100,23,23,23,3,404]))
print(o.solve([7]))
print(o.solve([7,6,9,6,9,6,9,7]))

3
0
1


# 8. Jump Game V

https://leetcode.com/problems/jump-game-v/

Array `arr` has integers and `d` is the max jump-size. You can jump anywhere between `i-d` and `i+d` from `i`, without exiting the array. Also you can jump to `j` only if for all `k` between `i` and `j`, `arr[j] < arr[i]` and `arr[k] < arr[i]`.

Starting from ANY index in `arr`, what is the max number of jumps you can make?

So, no concept of back-markers is necessary -- by definition, the jump has to be one-way, or not at all. Also, you should check and expand from `i`, outwards, because if you cannot jump to `i+1`, then you obviously cannot jump to `i+2`. 

Recursively, if I am at `i`, then I need a lookup that tells me if I can jump to within `i+-2`.

Then, I can do `1 + jumps(i+x)` for all possible next hops, and consider the maximum number. Again, `jump` is also a DP -- if I have visited a node, I should store it's max-jumps.

In [150]:
from collections import deque

class JumpGameV:
    def solve(self, arr, d):
        canhop = [[False for _ in arr] for _ in arr]
        
        # [6,4,14,6,8,13,9,7,10,6,12]
        
        for i in range(len(arr)):
            for j in range(i+1, i+d+1):
                if j >= len(arr):
                    break
                    
                if arr[j] >= arr[i]:
                    break
                    
                if j == i+1:
                    if arr[j] >= arr[i]:
                        break
                    else:
                        canhop[i][j] = True
                else:
                    if canhop[i][j-1] and arr[j] < arr[i]:
                        canhop[i][j] = True
                    else:
                        break
                        
            for j in range(i-d, i)[::-1]:
                if j < 0:
                    break
                
                if arr[j] >= arr[i]:
                    break
                    
                if j == i-1:
                    if arr[j] >= arr[i]:
                        break
                    else:
                        canhop[i][j] = True
                else:
                    if canhop[i][j+1] and arr[j] < arr[i]:
                        canhop[i][j] = True
                    else:
                        break
                        
        for row in canhop:
            print([int(x) for x in row])

        # Run BFS on this.
        graph = {}
        for 

In [151]:
d = 2
list(range(3-2, 3))

[1, 2]

In [153]:
o = JumpGameV()
print(o.solve([6,4,14,6,8,13,9,7,10,6,12], 2))
print(o.solve([7,6,5,4,3], 1))

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
3
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
5


# 9. Jump Game 6

You have an array of numbers, and you start from index 0. You can jump upto `k` positions everytime. What is the maximum possible score you can get till you reach the last index?

```
[1, -1, 2, 4, -7, 3], 2 => 7
```

In [176]:
import heapq

class JumpGameVI:
    def solve(self, nums, k):
        scores = [nums[0]]
        tracker = [(-nums[0], 0)]
        
        for ix in range(1, len(nums)):
            n = nums[ix]
            print(scores)
            while tracker and tracker[0][1] < ix-k:
                heapq.heappop(tracker)
                
            m = -tracker[0][0]
            scores.append(n + m)
            heapq.heappush(tracker, (-scores[-1], ix))
            
        print(scores)
        return scores[-1]

In [177]:
o = JumpGameVI()
o.solve([1, -1, -2, 4, -7, 3], 2)

[1]
[1, 0]
[1, 0, -1]
[1, 0, -1, 4]
[1, 0, -1, 4, -3]
[1, 0, -1, 4, -3, 7]


7

# 9 (alt).  Jump Game 6 -- `O(n)` solution

This tries to solve in **Linear Time**.

Instead of using a heap, use a Queue:
* Drop items on the left whose index has already passed.
* When you see a new number, add it to the queue *from the left* in a decreasing fashion:
    * Keep popping the items which are smaller, and *then* add this one.
    
**This only works because you need to keep track of the *LARGEST* item.**

In [178]:
from collections import deque

class JumpGameVIAlt:
    def solve(self, nums, k):
        q = deque([(nums[0], 0)])
        for ix in range(1, len(nums)):
            while q and q[0][1] < ix-k:
                q.popleft()
                
            m = q[0][0]
            v = m + nums[ix]
            while q and q[-1][0] < v:
                q.pop()
            
            q.append((v, ix))
            
        return q[0][0]

In [180]:
o = JumpGameVIAlt()
o.solve([1, -1, -2, 4, -7, 3], 2)

7

# 10. Jump Game 7

https://leetcode.com/problems/jump-game-vii/

Binary string, you start from `0`. You have `minJump` and `maxJump` -- your jumps have to be between these two limits, and you can only jump to another `0`. Return `True` if you can reach the end.

```
"011010", min=2, max=3 => True

"01101110", min=2, max=3 => False
```

In [181]:
class JumpGameVII:
    def solve(self, s, minJump, maxJump):
        canReachEnd = [False for _ in s]
        if s[-1] == '0':
            canReachEnd[-1] = True
        else:
            return False
        
        for j in range(len(s)-1, -1, -1):
            # `j` is where you have `jumped` to,
            # and you need to figure out if you can find a previous position
            # `i`, which is 0, and which falls between the min-max jumps.
            if s[j] == '1' or canReachEnd[j] is False:
                continue
                
            for i in range(j-maxJump, j-minJump+1):
                if i >= 0:
                    if s[i] == '0':
                        canReachEnd[i] = True
                        
        return canReachEnd[0]

In [184]:
o = JumpGameVII()
print(o.solve("011010", 2, 3))
print(o.solve("01101110", 2, 3))

True
False
